In [ ]:
import os
import tifffile
from aicsimageio import AICSImage
from aicsimageio.readers import CziReader
from scipy.ndimage import median_filter
from skimage.filters import threshold_otsu
from skimage.feature import canny
from skimage.measure import regionprops, label
from skimage.transform import rotate
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import hough_line, hough_line_peaks, rotate as skimage_rotate
from skimage.measure import label, regionprops
import tifffile
import xml.etree.ElementTree as ET
from scipy.ndimage import shift as nd_shift
import pandas as pd
from scipy.interpolate import UnivariateSpline
from scipy.ndimage import gaussian_filter1d


In [ ]:
def translate_and_rotate_image(image, angle, center_of_mass):
  
    image_center = np.array([image.shape[0] / 2, image.shape[1] / 2])
    translation = image_center - np.array(center_of_mass)

    
    translated_image = nd_shift(image, shift=translation, mode='constant', cval=0)

  
    rotated_image = rotate(translated_image, angle, resize=True, mode='constant', cval=0)

    return rotated_image

def rotate_all_channels(image_stack, angle, center_of_mass):
    rotated_stack = np.array([translate_and_rotate_image(image, angle, center_of_mass) for image in image_stack])
    return rotated_stack

def transpose_all_channels(image_stack):
    transposed_stack = np.array([np.transpose(image) for image in image_stack])
    return transposed_stack


def get_scale_from_metadata(metadata):

    root = ET.fromstring(metadata)


    scaling = root.find('.//Scaling/Items')

 
    if scaling is not None:
        scaling_x = float(scaling.find('.//Distance[@Id="X"]/Value').text) * 1e6
        scaling_y = float(scaling.find('.//Distance[@Id="Y"]/Value').text) * 1e6
    else:
        scaling_x = None
        scaling_y = None

    return scaling_x, scaling_y

def high_pass_filter(image, intensity_fraction):

    threshold_value = intensity_fraction * image.max()

    filtered_image = np.where(image > threshold_value, image, 0)

    return filtered_image

def extract_intensity_profile(image, center):
    y_center, x_center = center
    profile_length = image.shape[2] 
    mean_intensities = []

    for x in range(x_center, profile_length):
        
        slice_ = image[:, int(y_center-15):int(y_center+15), x]
        mean_intensity = slice_.mean(axis=1) 
        mean_intensities.append(mean_intensity)

    
    mean_intensities = np.array(mean_intensities)
    
    
    normalized_intensities = (mean_intensities - mean_intensities.min(axis=0)) / \
                             (mean_intensities.max(axis=0) - mean_intensities.min(axis=0))
    
    
    distances = np.arange(x_center, profile_length) * 1

    
    cutoff_index = np.where((normalized_intensities < 0.05).all(axis=1))[0]
    if cutoff_index.size > 0:
        distances = distances[:cutoff_index[0]]
        mean_intensities = mean_intensities[:cutoff_index[0]]

    return distances, mean_intensities

def process_image_for_profile(tiff_path, channel_num):
    with tifffile.TiffFile(tiff_path) as tif:
       
        if len(tif.pages) > 1:
            
            image_data = np.stack([page.asarray() for page in tif.pages])
        else:
            
            image_data = tif.asarray()
    
   
    center_of_mass = (image_data.shape[1] // 2, image_data.shape[2] // 2)
    
    
    distances, profiles = extract_intensity_profile(image_data[:channel_num], center_of_mass)
    distances = distances - distances[0]

   
    df = pd.DataFrame(profiles, index=distances, columns=[f'Channel {i}' for i in range(channel_num)])
    

    csv_path = tiff_path.replace('.tiff', '_profile.csv')
    df.to_csv(csv_path)
    
    return df

def process_czi_files(folder_path, center_ref_channel, edge_ref_channel, cannysigma=1.0):
    
    output_folder = f"{folder_path}_Rot"
    os.makedirs(output_folder, exist_ok=True)



    czi_files = [f for f in os.listdir(folder_path) if f.endswith('.czi')]

    for file_name in czi_files:
        
        file_path = os.path.join(folder_path, file_name)

       
        image = AICSImage(file_path)
        metadata = image.metadata
        
        image_data = image.get_image_data().squeeze()
      
        filtered_images = np.array([median_filter(image_data[i], size=3) for i in range(image_data.shape[0])])

       
        thresh_center = threshold_otsu(filtered_images[center_ref_channel])
        binary_center = filtered_images[center_ref_channel] > thresh_center
        labeled_center = label(binary_center)
        regions = regionprops(labeled_center)
       
        largest_region = max(regions, key=lambda r: r.area)
        center_of_mass = largest_region.centroid

       
        high_passed_image = high_pass_filter(filtered_images[edge_ref_channel], 0.25)
        thresh_edge = threshold_otsu(high_passed_image)
        binary_edge = filtered_images[edge_ref_channel] > thresh_edge
        edges = canny(binary_edge, sigma=cannysigma)
        tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
        h, theta, d = hough_line(edges, theta=tested_angles)
        _, angle, dist = next(zip(*hough_line_peaks(h, theta, d)))

        
      
        rotation_angle_degrees = np.rad2deg(angle) % 360
        rotated_images = rotate_all_channels(filtered_images, rotation_angle_degrees, center_of_mass)

        output_file_path = os.path.join(output_folder, file_name.replace('.czi', '.tiff'))

     
        with tifffile.TiffWriter(output_file_path, bigtiff=True) as tiff_writer:
            for i in range(rotated_images.shape[0]):
                
                tiff_writer.save(rotated_images[i])

        
        num_channels = rotated_images.shape[0]
        fig, axes = plt.subplots(1, num_channels, figsize=(num_channels * 5, 5))
        for i in range(num_channels):
            ax = axes[i] if num_channels > 1 else axes
            ax.imshow(rotated_images[i], cmap='gray')
            ax.set_title(f'Channel {i}')
            ax.axis('off')
        plt.suptitle(f'Rotated Channels for {file_name}')
        plt.tight_layout()
        plt.show()

In [ ]:

folder_path = 'xxx'  

center_ref_channel = 2 
edge_ref_channel = 1 
process_czi_files(folder_path, center_ref_channel, edge_ref_channel)

In [ ]:
# Example Usage
def plot_intensity_profiles(df, tiff_filename):
    # Plot each channel
    for column in df.columns:
        plt.plot(df.index, df[column], label=column)

    plt.xlabel('Distance (pixels)')
    plt.ylabel('Normalized Intensity')
    plt.title(f'Intensity Profile for {tiff_filename}')
    plt.legend()
    plt.show()


folder_path = 'xxx_Rot'  
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tiff'):
        tiff_path = os.path.join(folder_path, file_name)
        profile_df = process_image_for_profile(tiff_path, 3)
        if profile_df is not None:
            plot_intensity_profiles(profile_df, file_name)

In [ ]:
def process_file(file_path):

    data = pd.read_csv(file_path)

   
    center_region = data.iloc[0:10]
    avg_intensities = center_region[['Channel 0', 'Channel 1', 'Channel 2']].mean()


    for channel in ['Channel 0', 'Channel 1', 'Channel 2']:
        data[channel] = data[channel] - avg_intensities[channel]

    return data


def batch_process(folder_path, processed_folder):
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            data = process_file(file_path)
            processed_filename = filename.split('.csv')[0] + '_processed.csv'
            data.to_csv(os.path.join(processed_folder, processed_filename), index=False)

folder_path = 'xxx_Rot'
processed_folder = 'xxx_csv'
batch_process(folder_path, processed_folder)

In [ ]:

def smooth_data(data, sigma=10):
    """ Apply Gaussian smoothing to a data series. """
    return gaussian_filter1d(data, sigma)

def ThresholdMethod(data, channel, target_intensity):
    """ Find the first point where the channel intensity reaches the target intensity. """
    data[channel] = (data[channel] - data[channel].min()) / (data[channel].max() - data[channel].min())
    return np.where(data[channel] >= target_intensity)[0][0]

def TangentMethod(data, channel, percentage):
    channel_data = data[channel]
    half_max_value = percentage * channel_data.max()
    
    half_max_index = np.argmax(channel_data >= half_max_value)
    
   
    window_size = 3 
    start_index = max(0, half_max_index - window_size)
    end_index = min(len(channel_data), half_max_index + window_size + 1) 
    
    x = np.arange(start_index, end_index)
    y = channel_data[start_index:end_index]
    slope, intercept = np.polyfit(x, y, 1)
    
    intersection_point = -intercept / slope
    
    return int(intersection_point)


def calculate_info(df, time):
    df['Inferred_Time'] = (df['second_point']**3 - df['first_point']**3) / (df['third_point']**3 - df['first_point']**3) * 11
    df['Drug_Time'] = time
    return df

def analyze_profiles(folder_path, a, b, c):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            try:
                file_path = os.path.join(folder_path, filename)
                data = pd.read_csv(file_path)
                
                for channel in data.columns[1:]:  
                    data[channel] = smooth_data(data[channel])
                file_path = os.path.join(folder_path, filename)
                data = pd.read_csv(file_path)

                first_point = TangentMethod(data, 'Channel 3', a)
                second_point = TangentMethod(data, 'Channel 1', b)
                third_point = TangentMethod(data, 'Channel 2', c)
                forth_point = ThresholdMethod(data, 'Channel 2', 0.99)

                results.append({
                    'filename': filename,
                    'first_point': first_point,
                    'second_point': second_point,
                    'third_point': third_point,
                    'forth_point': forth_point
                })
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue  

    return pd.DataFrame(results)


t = analyze_profiles('folder_path', 0.5, 0.5, 0.5)
Drug_Time = 1
t_translate = calculate_info(t, Drug_Time)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import ptitprince as pt
import pandas as pd
import pandas as pd
import numpy as np
import os
from scipy.stats import kruskal
import scikit_posthocs as sp

df = pd.read_csv('/Users/eugeneyan/Desktop/genimi_Data&Code/Fig2/DyeSwitch(Fig2ij)/Source2ij/DyeSwitch_RainCloud.csv')


cloud_color = "grey" 
rain_color = "grey"   


f, ax = plt.subplots(figsize=(12, 8))
pt.RainCloud(x = 'Drug_Time', y='Inferred_Time', data=df, 
             palette=[rain_color],  box_showfliers=False, bw=0.2, width_viol = .7, ax = ax, orient = 'h' , alpha = .35, dodge = True)


for artist in ax.artists:
    artist.set_alpha(0.5)

for line in ax.lines:
    line.set_alpha(0.5)


for box in ax.patches:
    box.set_alpha(0.5)
    

ax.tick_params(axis='x', direction='in')
ax.tick_params(axis='y', direction='in')

plt.ylabel('Ground Truth (h)')
plt.xlabel('Inferred Time(h)')
plt.show()

In [ ]:


kruskal_result = kruskal(*[group["Inferred_Time"].values for name, group in df.groupby("Drug_Time")])

dunn_result = sp.posthoc_dunn(df, val_col="Inferred_Time", group_col="Drug_Time", p_adjust="bonferroni")


plt.figure(figsize=(10, 8))  
sns.heatmap(dunn_result, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Heatmap of Pairwise p-values from Dunn's Test")
plt.show()

In [ ]:
column_name = 'Inferred_Time'  
dataframe = df

data = [df[column_name].dropna() for df in dataframe]
std_devs = [df.std() for df in data]
std_devs_scaled = [df.std() * 60 for df in data]

labels = ['2', '3', '4', '5', '6', '7', '8']

filled_marker_style = dict(marker='o', linestyle=':', markersize=15,
                           color='darkgrey',
                           markerfacecolor='tab:blue',
                           markerfacecoloralt='lightsteelblue',
                           markeredgecolor='brown')

f, ax = plt.subplots(figsize=(12, 8))
plt.plot(labels, std_devs_scaled, color='black', marker='o', ms=5, mfc='w') 



ax.tick_params(axis='x', direction='in')
ax.tick_params(axis='y', direction='in')
plt.title('Standard Deviation of Inferred Times')
plt.xlabel('Ground Truth (h)', fontsize = 20)
plt.ylabel('Standard Deviation (min)')
plt.ylim(30, 120)
plt.grid(False)  

plt.show()

In [ ]:
data = df


data['time_difference'] = data['Inferred_Time'] - data['Drug_Time']
data['absolute_diff'] = np.abs(data['time_difference'])

plt.figure(figsize=(10, 6))

jittered_x = data['Drug_Time'] + np.random.uniform(-0.2, 0.2, size=len(data))


scatter = plt.scatter(
    x=jittered_x,
    y=data['absolute_diff'],
    c=data['absolute_diff'],
    cmap="coolwarm",
    alpha=0.8
)


cbar = plt.colorbar(scatter)
cbar.set_label('Time Difference (Color Coded)', fontsize=12)


plt.title('Time Difference', fontsize=14)
plt.xlabel('Drug Time', fontsize=12)
plt.ylabel('Time Difference (Inferred - Ground Truth)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(0, 5)


plt.show()